In [1]:
import sys;sys.path.append('d:/code/pywfn')

# 原子性质
所有的原子性质的计算器都在`pywfn.atomprop`子包下，其中的每一个模块封装了一种类型的原子性质的计算器

其包含的模块有
- `charge` 计算原子的各种电荷及自旋
- `activity` 计算原子的活性指标
- `direction` 计算以原子为中心的不同类型的方向向量。如：原子轨道方向、法向量方向、可能的反应方向等。
- `energy` 将分子轨道的能量分布到每个原子上。

每个模块下都有一个`Calculator`类，实例化时传入要计算的分子即可

## 原子电荷

说是电荷，其实直接计算得到的是每个原子上的电子数，用原子的核电荷数减去电子数即可得到原子电荷。

该计算器实例有个`form`属性，用以控制打印的结果时电子数量的形式还是电荷的形式，可以为`number`或`charge`

### Mulliken电荷

#### 计算公式
$$
q_A=Z_A-\sum_{\mu \in A}\sum_{\nu}P_{\mu\nu}S_{\mu\nu}
$$

#### 实例代码

In [2]:
from pywfn.base import Mole
from pywfn.reader import LogReader
from pywfn.atomprop import charge

path='mols/C6H6.out'
reader=LogReader(path)
mol=Mole(reader)
caler=charge.Calculator(mol)
caler.mulliken()

array([-0.1284, -0.1284, -0.1284, -0.1284, -0.1284, -0.1284,  0.1285,
        0.1285,  0.1285,  0.1285,  0.1285,  0.1285])

### lowdin电荷
#### 计算公式
$$
q_I = Z_I - \sum _{\mu \in I}(S^{1/2}·P·S^{1/2})
$$
#### 示例代码

In [3]:
from pywfn.base import Mole
from pywfn.reader import LogReader
from pywfn.atomprop import charge

path='mols/C6H6.out'
reader=LogReader(path)
mol=Mole(reader)
caler=charge.Calculator(mol)
caler.lowdin()

array([-0.1564, -0.1598, -0.1598, -0.1564, -0.1598, -0.1598,  0.1571,
        0.1595,  0.1595,  0.1571,  0.1595,  0.1595])

### hirshfeld电荷
#### 计算公式
#### 示例代码

In [4]:
from pywfn.base import Mole
from pywfn.reader import LogReader
from pywfn.atomprop import charge

path='mols/C6H6.out'
reader=LogReader(path)
mol=Mole(reader)
caler=charge.Calculator(mol)
caler.hirshfeld()

array([-0.0426, -0.0426, -0.0426, -0.0426, -0.0426, -0.0426,  0.0426,
        0.0427,  0.0427,  0.0426,  0.0427,  0.0427])

### 方向电子数

> 根据pocv方法，将原子的p轨道投影到不同方向得到投影系数矩阵，再以此带入原子电荷计算公式（Mulliken和Lowdin）得到的结果即为方向电子数，这里的方向需要自己指定。

#### 示例代码
`pywfn.atomprop.Calculator.dirElects`包含三个参数
```py
atms:list[int] # 需要进行投影操作的原子索引列表（从1开始）
dirs:np.ndarray # 对应原子需要投影到的方向，形状为[n,3]，n==len(atms)
ctype:str='mulliken' # 可以为 mulliken 或 lowdin
```

In [12]:
from pywfn.base import Mole
from pywfn.reader import LogReader
from pywfn.atomprop import charge
import numpy as np

path='mols/C6H6.out'
reader=LogReader(path)
mol=Mole(reader)
caler=charge.Calculator(mol)
dirs=np.random.rand(4,3)
dirs=dirs/np.linalg.norm(dirs,axis=1)[:,None] # 将方向向量归一化
atms=[1,2,3,4] # 
caler.dirElects(atms,dirs,'mulliken') # 我们对四个原子的轨道投影到随机的方向

array([0.711 , 0.7669, 0.7347, 0.6611, 0.    , 0.    , 0.    , 0.    ,
       0.    , 0.    , 0.    , 0.    ])

### π 电子数

指定每个原子的方向为其所在局部平面的法向量方向，没有法向量的则忽略（通常为饱和C原子），然后计算方向电子数，得到的结果即为 π 电子数

#### 示例代码

In [14]:
from pywfn.base import Mole
from pywfn.reader import LogReader
from pywfn.atomprop import charge

path='mols/C6H6.out'
reader=LogReader(path)
mol=Mole(reader)
caler=charge.Calculator(mol)
caler.piElects('mulliken') # 我们对四个原子的轨道投影到随机的方向

array([0.9851, 0.9851, 0.9851, 0.9851, 0.9851, 0.9851, 0.    , 0.    ,
       0.    , 0.    , 0.    , 0.    ])

In [15]:
caler.piDecomElects()

array([-0.1284, -0.1284, -0.1284, -0.1284, -0.1284, -0.1284,  0.1285,
        0.1285,  0.1285,  0.1285,  0.1285,  0.1285])

## 原子活性

原子活性就是分子中每个原子发生化学反应能力。

### 福井函数&双描述符
#### 计算公式
福井函数
$$f_A^+ = q_A^{N} - q_A^{N+1}$$
$$f_A^- = q_A^{N-1} - q_A^{N}$$
$$f_A^0 = (q_A^{N-1} - q_A^{N+1})/2$$

双描述符
$$\Delta f_A=f_A^+ - f_A^-$$

其中 $Q_A^{N+1}$ 是具有N+1个电子的分子的原子电荷，$Q_A^N$ 是具有N个电子的分子的原子电荷，$Q_A^{N-1}$ 是具有N-1个电子的分子的原子电荷。

- $f^+$ 被亲核试剂进攻的能力
- $f^-$ 被亲电试剂进攻的能力
- $f^0$ 发生自由基反应的能力
- $\Delta f$ 越正表示该位点越容易受到亲核试剂进攻
#### 示例代码

In [6]:
from pywfn.base import Mole
from pywfn.reader import LogReader
from pywfn.atomprop import activity

mol0=Mole(LogReader('./mols/C6H6.out'))
molN=Mole(LogReader('./mols/C6H6_N.out'))
molP=Mole(LogReader('./mols/C6H6_P.out'))

caler=activity.Calculator(mol0)
caler.fukui(molN,molP,'hirshfeld')

array([[-0.0426, -0.106 ,  0.1237,  0.1663,  0.0634,  0.1148, -0.1029],
       [-0.0426, -0.1734,  0.0436,  0.0862,  0.1309,  0.1085,  0.0447],
       [-0.0426, -0.1734,  0.0436,  0.0862,  0.1309,  0.1085,  0.0447],
       [-0.0426, -0.106 ,  0.1237,  0.1663,  0.0634,  0.1148, -0.1029],
       [-0.0426, -0.1734,  0.0436,  0.0862,  0.1309,  0.1085,  0.0447],
       [-0.0426, -0.1734,  0.0436,  0.0862,  0.1309,  0.1085,  0.0447],
       [ 0.0426, -0.0039,  0.104 ,  0.0614,  0.0465,  0.0539, -0.0149],
       [ 0.0427, -0.0217,  0.0925,  0.0498,  0.0643,  0.0571,  0.0145],
       [ 0.0427, -0.0217,  0.0925,  0.0498,  0.0643,  0.0571,  0.0145],
       [ 0.0426, -0.0039,  0.104 ,  0.0614,  0.0465,  0.0539, -0.0149],
       [ 0.0427, -0.0217,  0.0925,  0.0498,  0.0643,  0.0571,  0.0145],
       [ 0.0427, -0.0217,  0.0925,  0.0498,  0.0643,  0.0571,  0.0145]])

该段代码实例化计算器时也是传入一个N个电子的分子，调用函数的时候传入N+1个电子的分子、N个电子的分子及计算电荷的类型（这里使用Mulliken电荷）。

输出了7列数据，分别对应着 $q_{(N)}$，$q_{(N+1)}$，$q_{(N-1)}$，$f^-$，$f^+$，$f^0$，$\Delta f$